OBSERVACIONES GENERALES DE LA RESOLUCIÓN DE LA PRUEBA TÉCNICA


- Se han intentado evitar redundancias de return/ print en los métodos.

- En algunos métodos en los que se obtiene un valor numérico, como en los cálculos de ventas, se ha mantenido el return porque al tratarse de un valor numérico, tiene cierta lógica guardarlo para un uso futuro.

- En algunos métodos se ha añadido alguna característica adicional que puede ser interesante para su aclaración Se ha añadido en esa línea de código un comentario explicativo.

- Función adicional para no repetir código en los métodos (en el caso de strip y lower).

- Uso de list comprehension cuando su uso quedaba más sencillo y simplificado que el bucle for.

- Se ha utilizado una expresión del tipo regex en el caso del email.

- Instancias de clase y llamadas a métodos mediante inputs y secuencias de control.



A continuación se muestra la resolución de la prueba:

In [1]:
import re  #para validaciones regex en el caso del email
class TiendaOnline:

    def __init__(self):
        self.inventario = []
        self.ventas_totales = 0.0
        self.clientes = {} # diccionario de clientes


# Hacemos una función que vamos a aplicar en los métodos

    def limpiar(self, palabra):
        return palabra.lower().strip()
    

# Métodos referentes a los productos 
    def agregar_producto(self, nombre, precio, cantidad):
        for producto in self.inventario:
            if self.limpiar(producto['nombre']) == self.limpiar(nombre):
                producto['cantidad'] += cantidad
                return
        self.inventario.append({'nombre': self.limpiar(nombre), 'precio': precio, 'cantidad': cantidad}) 


    def ver_inventario(self): 
        if not self.inventario:
            print("No hay productos disponibles en este momento.")
            return #si no hay productos se sale del método
        
        print("Inventario de productos")
        for producto in self.inventario:
            print(f"- {producto['nombre']}: {producto['cantidad']} unidades, {producto['precio']:.2f} euros.")
        
        
    def buscar_producto(self, nombre):
        for producto in self.inventario:
            if self.limpiar(producto['nombre']) == self.limpiar(nombre):
                print(f"Producto encontrado {producto['nombre']}: {producto['cantidad']} unidades, {producto['precio']:.2f} euros")
                return #salimos del método después de imprimir. Sólo nos piden imprimir
        print(f"No se encontró el producto '{self.limpiar(nombre)}'.")
       

    def actualizar_stock(self, nombre, cantidad):   
        for producto in self.inventario:
            if self.limpiar(producto['nombre']) == self.limpiar(nombre):
                producto['cantidad'] = max(producto['cantidad'] + cantidad, 0) # evitamos stock negativo
                print(f"Stock de {producto['nombre']} se ha actualizado a {producto['cantidad']} unidades.")
                return #salimos del método después de atualizar
    
        print(f"No se encontró el producto {self.limpiar(nombre)}.")


    def eliminar_producto(self, nombre):
        for producto in self.inventario:
            if self.limpiar(producto['nombre']) == self.limpiar(nombre):
                self.inventario.remove(producto)
                print(f"El producto '{self.limpiar(nombre)}' ha sido eliminado.")
                return #salimos del método después de eliminar
        print(f"No se encontró el producto {self.limpiar(nombre)}.")


    def calcular_valor_inventario(self):
        total_inventario = sum(producto['cantidad'] * producto['precio'] for producto in self.inventario) # list comprehension para evitar bucle for más largo
        print(f"Valor total del inventario: {total_inventario:.2f} euros")
        return total_inventario #lo ponemos con objeto de que pueda guardarse y reutilizarse


# Métodos referentes a los clientes: compras, pagos, ventas

    def agregar_cliente(self, nombre_cliente, email):
        nombre_cliente = self.limpiar(nombre_cliente)
        email = email.lower().strip() # los emails se guardan en minúsculas

        # Validación email
        if not re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', email):
            print(f"El email '{email}' no tiene un formato válido.")
            return
       
        if nombre_cliente in self.clientes:
            print(f"El cliente '{nombre_cliente}' ya está registrado.")
            return #si está registrado el cliente salimos del método
        
        self.clientes[nombre_cliente] = {"email": email, "compras": []} 
        print(f"El cliente '{nombre_cliente}' se ha registrado correctamente.")


    def ver_clientes(self):   #si nos ajustamos al enunciado, solo se pide ver nombres y emails
        if not self.clientes:
            print("No hay clientes registrados.")
            return #salimos del método
        
        for nombre_cliente in sorted(self.clientes.keys()): # clientes ordenados por nombres
            detalles = self.clientes[nombre_cliente] 
            print(f"{nombre_cliente} ; Email: {detalles['email']}") #si solo queremos mostrar información, no es necesario return self.clientes
        


    def realizar_compra(self, nombre_cliente): #hemos decidido que este método se encargue del carrito
        nombre_cliente = self.limpiar(nombre_cliente)  
        if nombre_cliente not in self.clientes:
            print(f"El cliente {nombre_cliente} no está registrado.")
            return None
        
        carrito = {}
        while True:
            self.ver_inventario()
            producto_a_comprar = input("Producto a comprar o  'salir': ").strip()
            if producto_a_comprar.lower() == 'salir':
                break

            for producto in self.inventario: 
                if self.limpiar(producto['nombre']) == self.limpiar(producto_a_comprar):
                    try:
                        cantidad =abs(int(input(f"Cantidad de {producto['nombre']}: "))) #abs es porque no tendría ningún sentido introducir una cantidad negativa
                    except ValueError:
                        print("Cantidad inválida. Introduzca otro dato")
                        continue
            
                    if 0 < cantidad <= producto['cantidad']:
                        carrito[self.limpiar(producto['nombre'])] = {'precio': producto['precio'], 'cantidad': cantidad}
                        producto['cantidad'] -= cantidad
                    else:
                        print(f"No hay suficiente stock de producto {producto['nombre']}.")
                    break
            else:
                print(f"Producto {producto_a_comprar} no ha sido encontrado.")

        total_compra = sum(detalles['precio'] * detalles['cantidad'] for detalles in carrito.values())
        
        if carrito:
            print(f"El coste total de los productos en nuestro carrito es de: {total_compra:.2f} euros")

        return carrito 

    
    def procesar_pago(self, total_compra): #utilizamos un input dentro del método para ajustarnos a lo que pide el enunciado
        while True:
            try:
                cantidad_pago = float(input(f"Ingrese la cantidad a pagar ({total_compra:.2f} euros): ")) #por lógica es un float

                if cantidad_pago <= 0:
                    print("La cantidad pagada no puede ser negativa.")
                    continue

                dif = cantidad_pago - total_compra 

                if dif <0:
                    print(f"Faltan {-dif:.2f} € para completar el pago.")

                elif dif >0:
                    print(f"Pago recibido. Su cambio es {dif:.2f} euros.")
                    return dif
                
                else:
                    print("Pago exacto recibido.")
                    return 0

            except ValueError:
                print("Cantidad no válida, ingrese un número.")

    
    def registrar_compra(self, nombre_cliente, carrito): #guarda carrito, ventas totales
        nombre_cliente = self.limpiar(nombre_cliente)
        if nombre_cliente not in self.clientes:
            print(f"Cliente '{nombre_cliente}' no registrado. No se puede realizar la compra.")
            return #si el cliente no existe sale del método
        
        if not carrito:
            print("Carrito vacío. La compra no puede ser registrada")
            return
        
        total = sum(detalles['precio'] * detalles['cantidad'] for detalles in carrito.values())
        compra_datos = {'productos': carrito, 'total':total}

        self.clientes[nombre_cliente]['compras'].append(compra_datos)
        self.ventas_totales += total
        print(f"Compra registrada para '{nombre_cliente}' por un total de {total:.2f} euros.")

    

    def ver_compras_cliente(self, nombre_cliente):
        nombre_cliente = self.limpiar(nombre_cliente)
        if nombre_cliente not in self.clientes:
            print(f"Cliente '{nombre_cliente}' no se encuentra registrado.")
            return
        

        compras = self.clientes[nombre_cliente]['compras']
        
        if not compras:
            print(f"Cliente '{nombre_cliente}' no ha realizado ninguna compra.")
            return
        
        print(f"Historial de compras de {nombre_cliente}:") # se muestra el historial de compras del cliente por enunciado
        contador = 1 # 1, se refiere a la primera compra hecha

        for compra in compras:
            print(f"{contador}. Compra total: {compra['total']:.2f} euros.")
            for producto, detalle in compra['productos'].items():
                print(f"   - {producto}: {detalle['cantidad']} x {detalle['precio']:.2f} euros.")
            contador += 1 #se van sumando las compras



    def calcular_ventas_totales(self): # Simplemente devolvemos el valor acumulado en ventas_totales
        print(f"Las ventas totales de la tienda son: {self.ventas_totales:.2f} euros")
        return self.ventas_totales
    
  

In [ ]:
# Instancia de la clase
tienda = TiendaOnline()

while True:
    metodo = input(
        "\nIntroduce opción:\n"
        "1. Agregar producto\n"
        "2. Ver inventario\n"
        "3. Buscar producto\n"
        "4. Actualizar stock\n"
        "5. Eliminar producto\n"
        "6. Valor del inventario\n"
        "7. Agregar cliente\n"
        "8. Ver clientes\n"
        "9. Realizar compra\n"
        "10. Ver compras de cliente\n"
        "11. Calcular ventas totales\n"
        "12. Salir\n"
        "Opción: "
    ).strip()

    if metodo not in [str(i) for i in range (1,13)]:    #validación de las opciones
        print("Introduce una opción válida del 1 al 12.")
        continue


    if metodo == "1":
        try:
            precio = float(input("Precio del producto: "))
            cantidad = int(input("Cantidad del producto: "))
        except ValueError:
            print("Introduce un número válido para precio y cantidad.")
            continue
        nombre = input("Introduce el nombre del producto:")
        tienda.agregar_producto(nombre, precio, cantidad)

    elif metodo == "2":
        tienda.ver_inventario()

    elif metodo == "3":
        nombre = input("Nombre del producto a buscar: ")
        tienda.buscar_producto(nombre)

    elif metodo == "4":
        try:
            cantidad = int(input("Cantidad a sumar/restar: "))
        except ValueError:
            print("Introduce un número válido")
            continue
        nombre =input("Nombre del producto a actualizar:")
        tienda.actualizar_stock(nombre, cantidad)

    elif metodo == "5":
        nombre = input("Nombre del producto a eliminar: ")
        tienda.eliminar_producto(nombre)

    elif metodo == "6":
        tienda.calcular_valor_inventario()

    elif metodo == "7":
        nombre_cliente = input("Nombre del cliente: ")
        email = input("Email del cliente: ")
        tienda.agregar_cliente(nombre_cliente, email)

    elif metodo == "8":
        tienda.ver_clientes()

    elif metodo == "9":
        nombre_cliente = input("Nombre del cliente: ")
        carrito = tienda.realizar_compra(nombre_cliente)
        if carrito:
            total_compra = sum(detalle['precio'] * detalle['cantidad'] for detalle in carrito.values())
            print(f"\nResumen del carrito:")
            for producto, detalle in carrito.items():
                print(f"{producto}: {detalle['cantidad']} x {detalle['precio']:.2f} euros")
            tienda.procesar_pago(total_compra)
            tienda.registrar_compra(nombre_cliente, carrito)

    elif metodo == "10":
        nombre_cliente = input("Nombre del cliente: ")
        tienda.ver_compras_cliente(nombre_cliente)

    elif metodo == "11":
        tienda.calcular_ventas_totales()

    elif metodo == "12":
        print("Salir")
        break

